In [ ]:
pip install xlrd==1.2.0

In [ ]:
def creatingVertex(data):
    for index,row in data.iterrows():
        g.addV('PERSONA').\
        property(T.id,str(row["id"])).\
        property("name",row["name"]).\
        property("nickname", row["nickname"]).next()

def creatingEdge(checkUnion):
    for indexRow, row in enumerate(checkUnion):
        if not True in row:
            continue
        for indexColumn, column in enumerate(row):
            if column == True:
                client= g.V(str(int(indexRow + 1))).toList()[0]
                finalClient = g.V(str(int(indexColumn + 1))).toList()[0]
                
                g.V(client).addE('PERSONA').to(finalClient).property('weight',0.75).iterate()

In [ ]:
from __future__  import print_function  # Python 2/3 compatibility
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
import pandas as pd

graph = Graph()

neptuneEndpoint="your-neptune-endpoint"
remoteConn = DriverRemoteConnection(f'wss://{neptuneEndpoint}:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)

namesHeader = ["nickname", "id", "name"]
fileXLSX = "./Matriz_de_adyacencia_data_team.xlsx"

checkFile= False

try:
    listaDeActores = pd.read_excel(fileXLSX, "Lista de actores", header=2, names=namesHeader)
    matrizDeAdyacencia = pd.read_excel(fileXLSX, "Matriz de adyacencia", header=1)
    
    checkFile = True
    
except FileNotFoundError:
    checkFile = False

if checkFile:
    checkUnion = (matrizDeAdyacencia.values > 0 ).tolist()

    g.V().drop().iterate()

    creatingVertex(listaDeActores)
    creatingEdge(checkUnion)

    print(g.V().limit(100).toList()) # Showing all Vertex with the Id
    print(g.V().count().next()) # Showing the number of vertices 
    print(g.V().outE().elementMap().toList()) # Showing all the edges

else:
    print(f"The File called {fileXLSX} Doen't Exist ")

remoteConn.close()


In [ ]:
%%graph_notebook_config
{
  "host": "your-neptune-enpoint",
  "port": 8182,
  "auth_mode": "DEFAULT",
  "load_from_s3_arn": "",
  "ssl": true,
  "aws_region": "us-east-1a"
}


In [ ]:
%%gremlin -p v,ine,outv,oute,inv,oute,inv
g.V().hasLabel("PERSONA").inE().outV().outE().inV().outE().inV().path().by("name").by(label)